In [11]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import rowcol
from scipy.ndimage import distance_transform_edt
from shapely.geometry import Point
from tqdm import tqdm

Resample tif to be 3m perhaps?

In [ ]:
# === Parameters ===
input_ndvi_tif = "D:/terrain_generation_project/NAIP_processed/naip_tiles_1_8_redo_3_ndvi.tif"
structure_sf_poly = "D:/terrain_generation_project/structures/clipped_structure_polygons_altadena/clipped_structure_polygons_altadena.shp"

structure_points_gpkg = "D:/terrain_generation_project/structures_with_neighbors.gpkg"
structure_points_layer = "structures_with_neighbors"

# Outputs
output_layer = "structures_with_distance_ndvi"
output_polygons_gpkg = "D:/terrain_generation_project/NAIP_processed/structures_ndvi_poly_min_and_mean.gpkg"
output_points_gpkg = "D:/terrain_generation_project/NAIP_processed/structures_ndvi_points_min_and_mean.gpkg"

ndvi_min = 0.3
ndvi_max = 0.8

In [ ]:
# === Load NDVI and create vegetation mask ===
print("Loading NDVI raster...")
with rasterio.open(input_ndvi_tif) as src:
    ndvi = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata = src.nodata

    # Handle nodata
    valid_mask = (ndvi != nodata) if nodata is not None else np.ones_like(ndvi, dtype=bool)

    # NDVI mask: vegetation pixels between thresholds
    veg_mask = (ndvi >= ndvi_min) & (ndvi <= ndvi_max) & valid_mask
    inverse_mask = ~veg_mask

    print("Computing distance transform...")
    distance_pixels = distance_transform_edt(inverse_mask)

    # Convert to meters
    pixel_size = transform.a
    distance_meters = distance_pixels * pixel_size

# === Load structure polygons (Shapefile) and points (GeoPackage) ===
print("Loading structure polygons and points...")
structure_polys = gpd.read_file(structure_sf_poly).to_crs(crs)
structure_points = gpd.read_file(structure_points_gpkg, layer=structure_points_layer).to_crs(crs)

# === Spatial join: assign each point to a polygon ===
print("Joining points to polygons...")
joined = gpd.sjoin(structure_points, structure_polys, predicate="within", how="inner")

# Filter polygons to only those with matched points
structure_polys = structure_polys.loc[joined.index_right.unique()].copy()

# Prepare output columns
structure_polys["ndvi_min_dist_m"] = np.nan
structure_polys["ndvi_mean_dist_m"] = np.nan

# === Calculate distances from polygon boundaries ===
print("Calculating distances from polygons to vegetation (min and mean)...")
for idx, poly in tqdm(structure_polys.iterrows(), total=len(structure_polys), desc="Processing polygons"):
    try:
        boundary_coords = np.array(poly.geometry.boundary.coords)
        dists = []
        for x, y in boundary_coords:
            row, col = rowcol(transform, x, y)
            if (0 <= row < distance_meters.shape[0]) and (0 <= col < distance_meters.shape[1]):
                dists.append(distance_meters[row, col])

        if dists:
            structure_polys.at[idx, "ndvi_min_dist_m"] = np.min(dists)
            structure_polys.at[idx, "ndvi_mean_dist_m"] = np.mean(dists)
    except Exception:
        pass  # leave NaNs

# === Assign distances to points based on matched polygon ===
print("Assigning distances back to points...")
joined = joined.reset_index(drop=True)
joined = joined.merge(
    structure_polys[["ndvi_min_dist_m", "ndvi_mean_dist_m"]],
    left_on="index_right",
    right_index=True
)

# === Save to separate GeoPackages ===
print(f"Saving polygon output to: {output_polygons_gpkg} (layer: polygons)...")
structure_polys.to_file(output_polygons_gpkg, layer="polygons", driver="GPKG")

print(f"Saving point output to: {output_points_gpkg} (layer: points)...")
joined.to_file(output_points_gpkg, layer="points", driver="GPKG")

print("Done.")

In [ ]:
# # === Load NDVI and create vegetation mask ===
# print("Loading NDVI raster...")
# with rasterio.open(input_ndvi_tif) as src:
#     ndvi = src.read(1)
#     transform = src.transform
#     crs = src.crs
#     nodata = src.nodata

#     # Handle nodata
#     valid_mask = (ndvi != nodata) if nodata is not None else np.ones_like(ndvi, dtype=bool)

#     # NDVI mask
#     veg_mask = (ndvi >= ndvi_min) & (ndvi <= ndvi_max) & valid_mask
#     inverse_mask = ~veg_mask

#     print("Computing distance transform...")
#     distance_pixels = distance_transform_edt(inverse_mask)

#     # Convert to meters
#     pixel_size = transform.a
#     distance_meters = distance_pixels * pixel_size

# # === Load structure polygons (Shapefile) and points (GeoPackage) ===
# print("Loading structure polygons and points...")
# structure_polys = gpd.read_file(structure_sf_poly).to_crs(crs)
# structure_points = gpd.read_file(structure_points_gpkg, layer=structure_points_layer).to_crs(crs)

# # === Spatial join: assign each point to a polygon ===
# print("Joining points to polygons...")
# joined = gpd.sjoin(structure_points, structure_polys, predicate="within", how="inner")

# # Filter polygons to only those with matched points
# structure_polys = structure_polys.loc[joined.index_right.unique()].copy()
# structure_polys["ndvi_dist_m"] = np.nan  # Placeholder column

# # === Calculate distance for each polygon boundary ===
# print("Calculating distances from polygons to vegetation...")
# for idx, poly in tqdm(structure_polys.iterrows(), total=len(structure_polys), desc="Processing polygons"):
#     try:
#         boundary = np.array(poly.geometry.boundary.coords)
#         dists = []
#         for x, y in boundary:
#             row, col = rowcol(transform, x, y)
#             if (0 <= row < distance_meters.shape[0]) and (0 <= col < distance_meters.shape[1]):
#                 dists.append(distance_meters[row, col])
#         structure_polys.at[idx, "ndvi_dist_m"] = min(dists) if dists else np.nan
#     except Exception:
#         structure_polys.at[idx, "ndvi_dist_m"] = np.nan

# # === Assign distances to points based on matched polygon ===
# print("Assigning distances back to points...")
# joined = joined.reset_index(drop=True)
# joined = joined.merge(
#     structure_polys[["ndvi_dist_m"]], left_on="index_right", right_index=True
# )

# # === Save to separate GeoPackages ===
# print(f"Saving polygon output to: {output_polygons_gpkg} (layer: polygons)...")
# structure_polys.to_file(output_polygons_gpkg, layer="polygons", driver="GPKG")

# print(f"Saving point output to: {output_points_gpkg} (layer: points)...")
# joined.to_file(output_points_gpkg, layer="points", driver="GPKG")

# print("Done.")

Loading NDVI raster...
Computing distance transform...
Loading structure polygons and points...
Joining points to polygons...
Calculating distances from polygons to vegetation...


Processing polygons: 100%|██████████| 9231/9231 [00:04<00:00, 2086.26it/s]


Assigning distances back to points...
Saving polygon output to: D:/terrain_generation_project/NAIP_processed/structures_ndvi_poly.gpkg (layer: polygons)...
Saving point output to: D:/terrain_generation_project/NAIP_processed/structures_ndvi_points.gpkg (layer: points)...
Done.
